# Detect the bottom dots on an image and use it to construct a spatial mapping

## Set number of frames, bitdepth, particle size, filename of the measurements and filename of the calibration files

In [ ]:
# number of frames
N = 1850

# bitdepth
bitdepth = 16

# particle size (Large = 1, Medium/Small = 0)
Size = 0

# startframe number
startframe = 1


m = 5 #number of row in start calibration grid
n = 6 #number of columns in start calibration grid


# filename calibration file at the start
path ='C:/Users/s131431/Documents/Thesis/Opnames/2021-05-04/2021-05-04/2021-05-04/end_calibration.tiff'


# filename calibration file at the end
path_end ='C:/Users/s131431/Documents/Thesis/Opnames/2021-05-04/2021-05-04/2021-05-04/end_calibration.tiff'


#filename of the measurements
filesname = 'C:/Users/s131431/Documents/Thesis/Opnames/2021-05-04/2021-05-04/2021-05-04/curvelength714stroke40/measurement'
savename = '2021-05-04curvelength714stroke40.txt'

In [ ]:
# Load modules
#import cv2 
import numpy as np
import matplotlib.pyplot as plt
import cv2
import imutils
import statsmodels.api as sm
import pandas as pd

from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.dpi'] = 200

#from matplotlib.collections import LineCollection
#from matplotlib.colors import ListedColormap, BoundaryNorm




## Find the angle over which to rotate the end calibration


In [ ]:


# If image has bitdepth 8 or it has sharp contrasts making grayscale values obsolete
if bitdepth == 8:
    image = cv2.imread(path_end,cv2.CV_8UC1) * 16
# 
elif bitdepth == 16:
    image = cv2.imread(path_end,cv2.CV_16UC1)




# Fixed thresholding
meanvalue = int(np.mean(image))
thresholdvalue = meanvalue/1.5
binaryImage = cv2.threshold(image,thresholdvalue,255,cv2.THRESH_BINARY_INV)[1]


# Define a course kernel
kernelClosed = np.ones((7,7),np.uint8)
kernelOpen   = np.ones((5,5),np.uint8)

# Reduce noise inside particles
binaryImageClosed = cv2.morphologyEx(binaryImage, cv2.MORPH_CLOSE, kernelClosed)


# Reduce noise outside particles
binaryImageOpened = cv2.morphologyEx(binaryImageClosed, cv2.MORPH_OPEN, kernelOpen)
markers = cv2.connectedComponents(np.uint8(binaryImageOpened))[1]
Nmarkers = len(np.unique(markers))
Ndots = int(n*m)
maximumSize = 80
minimumSize = 30
xdots = np.empty(0,dtype=float)
ydots = np.empty(0,dtype=float)

for i in range(1,Nmarkers,1):
    idx = np.argwhere(markers==i)
    
    xSpread = np.max(idx[:,1])-np.min(idx[:,1])    
    ySpread = np.max(idx[:,0])-np.min(idx[:,0])
    
    if ((minimumSize < xSpread < maximumSize) and (minimumSize < ySpread < maximumSize)):
        xdots = np.append(xdots,np.mean(idx[:,1]))
        ydots = np.append(ydots,np.mean(idx[:,0]))
    

# Sort the dots
NRows = m
NColumns = n

for i in range(0,NRows,1):
    idx = np.argsort(xdots[i*NColumns:(i+1)*NColumns])
    
    xdots[i*NColumns:(i+1)*NColumns] = xdots[i*NColumns+idx]
    ydots[i*NColumns:(i+1)*NColumns] = ydots[i*NColumns+idx]
    
# Positions of the dots in meters
distanceDots = 0.02 # meters
x = np.linspace(0,(NColumns-1)*distanceDots,NColumns)
y = np.linspace(0,(NRows-1)   *distanceDots,NRows)
xReal, yReal = np.meshgrid(x,y)


Z = (ydots[(n-1)]-ydots[0])/(xdots[(n-1)]-xdots[0])

phi_end = np.arctan(Z)
phi_end = phi_end*180/np.pi

print(phi_end)

## Extract grid data from end calibration

In [ ]:

Rotation_end = phi_end

# Read image
image_end = cv2.imread(path_end)


# If image has bitdepth 8 or it has sharp contrasts making grayscale values obsolete
if bitdepth == 8:
    image_end = cv2.imread(path_end,cv2.CV_8UC1) * 16
# 
elif bitdepth == 16:
    image_end = cv2.imread(path_end,cv2.CV_16UC1)

# Show image

image_end = imutils.rotate(image_end, angle=Rotation_end)
#plt.figure()
#plt.imshow(image*1,origin='lower',cmap='gray')
#plt.show()
 

#plt.figure()
#plt.imshow(image*1,origin='lower',cmap='gray')
#plt.show()


# Fixed thresholding
meanvalue = int(np.mean(image_end))
thresholdvalue = meanvalue/1.5
binaryImage = cv2.threshold(image_end,thresholdvalue,255,cv2.THRESH_BINARY_INV)[1]


# Define a course kernel
kernelClosed = np.ones((7,7),np.uint8)
kernelOpen   = np.ones((5,5),np.uint8)

# Reduce noise inside particles
binaryImageClosed = cv2.morphologyEx(binaryImage, cv2.MORPH_CLOSE, kernelClosed)


# Reduce noise outside particles
binaryImageOpened = cv2.morphologyEx(binaryImageClosed, cv2.MORPH_OPEN, kernelOpen)
markers = cv2.connectedComponents(np.uint8(binaryImageOpened))[1]
Nmarkers = len(np.unique(markers))
Ndots = int(n*m)      # change for the number of dots on the calibration grid
maximumSize = 80
minimumSize = 30
xdots = np.empty(0,dtype=float)
ydots = np.empty(0,dtype=float)

for i in range(1,Nmarkers,1):
    idx = np.argwhere(markers==i)
    
    xSpread = np.max(idx[:,1])-np.min(idx[:,1])    
    ySpread = np.max(idx[:,0])-np.min(idx[:,0])
    
    if ((minimumSize < xSpread < maximumSize) and (minimumSize < ySpread < maximumSize)):
        xdots = np.append(xdots,np.mean(idx[:,1]))
        ydots = np.append(ydots,np.mean(idx[:,0]))
    

   
    
plt.figure()
plt.imshow(image_end,origin='lower',cmap='gray')
plt.plot(xdots,ydots,'r.',markersize=1)
plt.show()

# Sort the dots
NRows = m
NColumns = n

for i in range(0,NRows,1):
    idx = np.argsort(xdots[i*NColumns:(i+1)*NColumns])
    
    xdots[i*NColumns:(i+1)*NColumns] = xdots[i*NColumns+idx]
    ydots[i*NColumns:(i+1)*NColumns] = ydots[i*NColumns+idx]
    
# Positions of the dots in meters
distanceDots = 0.02 # meters
x = np.linspace(0,(NColumns-1)*distanceDots,NColumns)
y = np.linspace(0,(NRows-1)   *distanceDots,NRows)
xReal, yReal = np.meshgrid(x,y)

# We need these two arrays as input data sets for the least square fitting
Bx = xReal.flatten()
By = yReal.flatten()

# Positions of the dots in pixels
X_end = xdots*1 -xdots[0]
Y_end = ydots*1 -ydots[0]
A_end     = np.array([X_end*0+1, X_end, Y_end, X_end**2, X_end**2*Y_end, X_end**2*Y_end**2, Y_end**2, X_end*Y_end**2, X_end*Y_end]).T

# Use linear least square fitting to find the coefficients that solve A*x=B
coeffxx, _, _, _ = np.linalg.lstsq(A_end,Bx,rcond=None)
coeffyy, _, _, _ = np.linalg.lstsq(A_end,By,rcond=None)

# This is the function we just used for fitting, but here explicit
def surface(x,y, a,b,c,d,e,f,g,h,i):
    return a + b*x + c*y + d*x**2 + e*x**2*y + f*x**2*y**2 + g*y**2 + h*x*y**2 + i*x*y

# This is the function to obtain the physical coordinates from the pixel coordinates
def conversion_end(X,Y):
    X1 = surface(X,Y,*coeffxx)
    Y1 = surface(X,Y,*coeffyy)
    return (X1,Y1)
def xconversion_end(X,Y):
    X1 = surface(X,Y,*coeffxx)
    Y1 = surface(X,Y,*coeffyy)
    return(X1)
def yconversion_end(X,Y):
    X1 = surface(X,Y,*coeffxx)
    Y1 = surface(X,Y,*coeffyy)
    return(Y1)




## Plot and compare datapoints with fitted surfaces
#xplot,yplot = np.meshgrid(np.linspace(0,1500,100),np.linspace(0,1500,100))

#xplot = xplot.flatten()
#yplot = yplot.flatten()

#fig = plt.figure()
#ax = fig.add_subplot(111,projection='3d')
#ax.scatter(X_end,Y_end,xReal)
#ax.plot_trisurf(xplot,yplot,surface(xplot,yplot,*coeffxx),alpha=0.3)
#ax.set_xlabel('x img [pixels]')
#ax.set_ylabel('y img [pixels]')
#ax.set_zlabel('x real [m]')
#plt.show()

#fig = plt.figure()
#ax = fig.add_subplot(111,projection='3d')
#ax.scatter(X_end,Y_end,yReal)
#ax.plot_trisurf(xplot,yplot,surface(xplot,yplot,*coeffyy),alpha=0.3)
#ax.set_xlabel('x img [pixels]')
#ax.set_ylabel('y img [pixels]')
#ax.set_zlabel('y real [m]')
#plt.show()

## Find the angle over which to rotate the start calibration image

In [ ]:
# If image has bitdepth 8 or it has sharp contrasts making grayscale values obsolete
if bitdepth == 8:
    image = cv2.imread(path,cv2.CV_8UC1) * 16
# 
elif bitdepth == 16:
    image = cv2.imread(path,cv2.CV_16UC1)


# Fixed thresholding
meanvalue = int(np.mean(image))
thresholdvalue = meanvalue/1.5
binaryImage = cv2.threshold(image,thresholdvalue,255,cv2.THRESH_BINARY_INV)[1]


# Define a course kernel
kernelClosed = np.ones((7,7),np.uint8)
kernelOpen   = np.ones((5,5),np.uint8)

# Reduce noise inside particles
binaryImageClosed = cv2.morphologyEx(binaryImage, cv2.MORPH_CLOSE, kernelClosed)


# Reduce noise outside particles
binaryImageOpened = cv2.morphologyEx(binaryImageClosed, cv2.MORPH_OPEN, kernelOpen)
markers = cv2.connectedComponents(np.uint8(binaryImageOpened))[1]
Nmarkers = len(np.unique(markers))
Ndots = int(n*m)
maximumSize = 80
minimumSize = 30
xdots = np.empty(0,dtype=float)
ydots = np.empty(0,dtype=float)

for i in range(1,Nmarkers,1):
    idx = np.argwhere(markers==i)
    
    xSpread = np.max(idx[:,1])-np.min(idx[:,1])    
    ySpread = np.max(idx[:,0])-np.min(idx[:,0])
    
    if ((minimumSize < xSpread < maximumSize) and (minimumSize < ySpread < maximumSize)):
        xdots = np.append(xdots,np.mean(idx[:,1]))
        ydots = np.append(ydots,np.mean(idx[:,0]))
    

# Sort the dots
NRows = m
NColumns = n

for i in range(0,NRows,1):
    idx = np.argsort(xdots[i*NColumns:(i+1)*NColumns])
    
    xdots[i*NColumns:(i+1)*NColumns] = xdots[i*NColumns+idx]
    ydots[i*NColumns:(i+1)*NColumns] = ydots[i*NColumns+idx]
    

Z2 = (ydots[n-1]-ydots[0])/(xdots[n-1]-xdots[0])

phi = np.arctan(Z2)
phi = phi*180/np.pi

print(phi)

## Extract grid data from start calibration and convert into physical coordinates

In [ ]:


# Read image
image = cv2.imread(path)


Rotation = phi

# If image has bitdepth 8 or it has sharp contrasts making grayscale values obsolete
if bitdepth == 8:
    image = cv2.imread(path,cv2.CV_8UC1) * 16
# 
elif bitdepth == 16:
    image = cv2.imread(path,cv2.CV_16UC1)

# Show image
image = imutils.rotate(image, angle=Rotation)
#plt.figure()
#plt.imshow(image*1,origin='lower',cmap='gray')
#plt.show()


#plt.figure()
#plt.imshow(image*1,origin='lower',cmap='gray')
#plt.show()

# Fixed thresholding
meanvalue = int(np.mean(image))
thresholdvalue = meanvalue/1.5
binaryImage = cv2.threshold(image,thresholdvalue,255,cv2.THRESH_BINARY_INV)[1]

# Define a course kernel
kernelClosed = np.ones((7,7),np.uint8)
kernelOpen   = np.ones((5,5),np.uint8)

# Reduce noise inside particles
binaryImageClosed = cv2.morphologyEx(binaryImage, cv2.MORPH_CLOSE, kernelClosed)

# Reduce noise outside particles
binaryImageOpened = cv2.morphologyEx(binaryImageClosed, cv2.MORPH_OPEN, kernelOpen)
markers = cv2.connectedComponents(np.uint8(binaryImageOpened))[1]
Nmarkers = len(np.unique(markers))
Ndots = int(n*m)     #change for the number of dots on the calibration grid
maximumSize = 80
minimumSize = 30
xdots = np.empty(0,dtype=float)
ydots = np.empty(0,dtype=float)

for i in range(1,Nmarkers,1):
    idx = np.argwhere(markers==i)
    
    xSpread = np.max(idx[:,1])-np.min(idx[:,1])    
    ySpread = np.max(idx[:,0])-np.min(idx[:,0])
    
    if ((minimumSize < xSpread < maximumSize) and (minimumSize < ySpread < maximumSize)):
        xdots = np.append(xdots,np.mean(idx[:,1]))
        ydots = np.append(ydots,np.mean(idx[:,0]))
    
plt.figure()
plt.imshow(image,origin='lower',cmap='gray')
plt.plot(xdots,ydots,'r.',markersize=1)
plt.show()

# Sort the dots
NRows = m
NColumns = n

for i in range(0,NRows,1):
    idx = np.argsort(xdots[i*NColumns:(i+1)*NColumns])
    
    xdots[i*NColumns:(i+1)*NColumns] = xdots[i*NColumns+idx]
    ydots[i*NColumns:(i+1)*NColumns] = ydots[i*NColumns+idx]
    
# Positions of the dots in meters
distanceDots = 0.02 # meters
x = np.linspace(0,(NColumns-1)*distanceDots,NColumns)
y = np.linspace(0,(NRows-1)   *distanceDots,NRows)
xReal, yReal = np.meshgrid(x,y)

# We need these two arrays as input data sets for the least square fitting
Bx = xReal.flatten()
By = yReal.flatten()

# Positions of the dots in pixels
X = xdots*1 - xdots[0]
Y = ydots*1 - ydots[0]
A = np.array([X*0+1, X, Y, X**2, X**2*Y, X**2*Y**2, Y**2, X*Y**2, X*Y]).T

# Use linear least square fitting to find the coefficients that solve A*x=B
coeffx, _, _, _ = np.linalg.lstsq(A,Bx,rcond=None)
coeffy, _, _, _ = np.linalg.lstsq(A,By,rcond=None)

# This is the function we just used for fitting, but here explicit
def surface(x,y, a,b,c,d,e,f,g,h,i):
    return a + b*x + c*y + d*x**2 + e*x**2*y + f*x**2*y**2 + g*y**2 + h*x*y**2 + i*x*y

# This is the function to obtain the physical coordinates from the pixel coordinates
def conversion(X,Y):
    X1 = surface(X,Y,*coeffx)
    Y1 = surface(X,Y,*coeffy)
    return (X1,Y1)

def xconversion(X,Y):
    X1 = surface(X,Y,*coeffx)
    Y1 = surface(X,Y,*coeffy)
    return(X1)    
def yconversion(X,Y):
    X1 = surface(X,Y,*coeffx)
    Y1 = surface(X,Y,*coeffy)
    return(Y1)




## Plot and compare datapoints with fitted surfaces
#xplot,yplot = np.meshgrid(np.linspace(0,1500,100),np.linspace(0,1500,100))
#xplot = xplot.flatten()
#yplot = yplot.flatten()

#fig = plt.figure()
#ax = fig.add_subplot(111,projection='3d')
#ax.scatter(X,Y,xReal)
#ax.plot_trisurf(xplot,yplot,surface(xplot,yplot,*coeffx),alpha=0.3)
#ax.set_xlabel('x img [pixels]')
#ax.set_ylabel('y img [pixels]')
#ax.set_zlabel('x real [m]')
#plt.show()

#fig = plt.figure()
#ax = fig.add_subplot(111,projection='3d')
#ax.scatter(X,Y,yReal)
#ax.plot_trisurf(xplot,yplot,surface(xplot,yplot,*coeffy),alpha=0.3)
#ax.set_xlabel('x img [pixels]')
#ax.set_ylabel('y img [pixels]')
#ax.set_zlabel('y real [m]')
#plt.show()



# Particle recognition

## Find the pixel coordinates for both particles for every frame

In [ ]:
# Pixel data
ParticlePixelx1 = np.zeros(N)
ParticlePixely1 = np.zeros(N)
ParticlePixelx2 = np.zeros(N)
ParticlePixely2 = np.zeros(N)
    
    
if   Size == 1:
        maximumSize = 500
elif Size == 0:
        maximumSize = 80
        
#maximumSize = 50
minimumSize = 5


# Read image
for i in range(0,N,1):
    l = i
    i = i+ startframe
    x = str(i)
    x = x.zfill(4)
    filename = filesname + x + '.tiff'

    image = cv2.imread(filename)
    image = cv2.imread(filename,cv2.CV_16UC1)
    
           
    # Fixed thresholding
    meanvalue = int(np.mean(image))
    thresholdvalue = meanvalue/1.9
    binaryImage = cv2.threshold(image,thresholdvalue,255,cv2.THRESH_BINARY_INV)[1]

    # Define a course kernel
    kernelClosed = np.ones((1,1),np.uint8)
    kernelOpen   = np.ones((1,1),np.uint8)

    # Reduce noise inside particles
    binaryImageClosed = cv2.morphologyEx(binaryImage, cv2.MORPH_CLOSE, kernelClosed)

    # Reduce noise outside particles
    binaryImageOpened = cv2.morphologyEx(binaryImageClosed, cv2.MORPH_OPEN, kernelOpen)

    markers = cv2.connectedComponents(np.uint8(binaryImageOpened))[1]
  


    
    
    Nmarkers = len(np.unique(markers))
    Ndots = int(2)
    

    xdots = np.empty(0,dtype=float)
    ydots = np.empty(0,dtype=float)

    for k in range(1,Nmarkers,1):
        idx = np.argwhere(markers==k)
    
        xSpread = np.max(idx[:,1])-np.min(idx[:,1])    
        ySpread = np.max(idx[:,0])-np.min(idx[:,0])
    
        if ((minimumSize < xSpread < maximumSize) and (minimumSize < ySpread < maximumSize)):
            xdots = np.append(xdots,np.mean(idx[:,1]))
            ydots = np.append(ydots,np.mean(idx[:,0]))
    

    ParticlePixelx1[l] = xdots[0]
    ParticlePixely1[l] = ydots[0]
    ParticlePixelx2[l] = xdots[1]
    ParticlePixely2[l] = ydots[1] 


       
# check the last frame    
plt.figure()
plt.imshow(image,origin='lower',cmap='gray')
plt.plot(xdots,ydots,'r.',markersize=1)
plt.grid()
plt.show()


## Convert pixel coordinates to physical coordinates using the start calibration and plot the data

In [ ]:
# Physical coordinates from the start calibration
Particlex1 = np.zeros(N)
Particley1 = np.zeros(N)
Particlex2 = np.zeros(N)
Particley2 = np.zeros(N)


#x_correction_end = xconversion_end(X_end[0],Y_end[0])
#y_correction_end = yconversion_end(X_end[0],Y_end[0])
#x_correction = xconversion(X[0],Y[0])
#y_correction = yconversion(X[0],Y[0])


Particlex1 = xconversion(ParticlePixelx1, ParticlePixely1) #- x_correction
Particley1 = yconversion(ParticlePixelx1, ParticlePixely1) #- y_correction
Particlex2 = xconversion(ParticlePixelx2, ParticlePixely2) #- x_correction
Particley2 = yconversion(ParticlePixelx2, ParticlePixely2) #- y_correction




#plt.figure()
#plt.plot(Particlex1,Particley1)
#plt.plot(Particlex1[0], Particley1[0], 'r.')
#plt.xlabel('x coordinate [m]')
#plt.ylabel('y coordinate [m]')
#plt.legend(['Particle1', 'Start point'])
#plt.grid()
#plt.show()


#plt.figure()
#plt.plot(Particlex2,Particley2)
#plt.plot(Particlex2[0], Particley2[0], 'r.')
#plt.xlabel('x coordinate [m]')
#plt.ylabel('y coordinate [m]')
#plt.legend(['Particle2', 'Start point'])
#plt.grid()
#plt.show()


plt.figure()
plt.plot(Particlex1,Particley1)
plt.plot(Particlex1[0], Particley1[0], 'r.')
plt.plot(Particlex2,Particley2)
plt.plot(Particlex2[0], Particley2[0], 'r.')
plt.xlabel('x coordinate [m]')
plt.ylabel('y coordinate [m]')
#plt.legend(['Particle1', 'Start point 1', 'Particle2','Start point 2'])
plt.grid()
plt.show()

## Check particles at the last frame

In [ ]:
# Fixed thresholding
meanvalue = int(np.mean(image))
thresholdvalue = meanvalue/1.5
binaryImage = cv2.threshold(image,thresholdvalue,255,cv2.THRESH_BINARY_INV)[1]

# Define a course kernel
kernelClosed = np.ones((1,1),np.uint8)
kernelOpen   = np.ones((1,1),np.uint8)

# Reduce noise inside particles
binaryImageClosed = cv2.morphologyEx(binaryImage, cv2.MORPH_CLOSE, kernelClosed)

# Reduce noise outside particles
binaryImageOpened = cv2.morphologyEx(binaryImageClosed, cv2.MORPH_OPEN, kernelOpen)

markers = cv2.connectedComponents(np.uint8(binaryImageOpened))[1]

Nmarkers = len(np.unique(markers))
Ndots = int(13*7)
maximumSize = 50
minimumSize = 25

xdots = np.empty(0,dtype=float)
ydots = np.empty(0,dtype=float)

for i in range(1,Nmarkers,1):
    idx = np.argwhere(markers==i)
    
    xSpread = np.max(idx[:,1])-np.min(idx[:,1])    
    ySpread = np.max(idx[:,0])-np.min(idx[:,0])
    
    if ((minimumSize < xSpread < maximumSize) and (minimumSize < ySpread < maximumSize)):
        xdots = np.append(xdots,np.mean(idx[:,1]))
        ydots = np.append(ydots,np.mean(idx[:,0]))
    
plt.figure()
plt.imshow(image,origin='lower',cmap='gray')
plt.plot(xdots,ydots,'r.',markersize=1)
plt.show()


plt.figure()
plt.imshow(binaryImage)
plt.show()

plt.figure()
plt.imshow(binaryImageClosed)
plt.show()

plt.figure()
plt.imshow(binaryImageOpened)
plt.show()



## Find difference between particle coordinates using the start and the end calibration

In [ ]:
# physical coordinates using the end calibration
Particlex1_end = xconversion_end(ParticlePixelx1, ParticlePixely1) #- x_correction_end
Particley1_end = yconversion_end(ParticlePixelx1, ParticlePixely1) #- y_correction_end
Particlex2_end = xconversion_end(ParticlePixelx2, ParticlePixely2) #- x_correction_end
Particley2_end = yconversion_end(ParticlePixelx2, ParticlePixely2) #- y_correction_end

#Particle1_end = conversion_end(ParticlePixelx1, ParticlePixely1)
#Particle2_end = conversion_end(ParticlePixelx2, ParticlePixely2)


#physical coordinates suing the start calibration
#Particle1 = np.zeros((2,N))
#Particle2 = np.zeros((2,N))

#Particle1 = conversion(ParticlePixelx1, ParticlePixely1)
#Particle2 = conversion(ParticlePixelx2, ParticlePixely2) 


# difference from start and end calibration
xdifference1 = Particlex1 - Particlex1_end 
ydifference1 = Particley1 - Particley1_end
xdifference2 = Particlex2 - Particlex2_end
ydifference2 = Particley2 - Particley2_end 

#ydifference1 = Particley1 - Particley1_end
#xdifference2 = Particle2[0] - Particle2_end[0]
#ydifference2 = Particle2[1] - Particle2_end[1]


#print(Particle1)
#print(Particle2)
#print(Particle1_end)
#print(Particle2_end)

print(np.mean(xdifference1))
print(np.mean(ydifference1))
print(np.mean(xdifference2))
print(np.mean(ydifference2))

In [ ]:
# fit plots
plt.figure()

plt.plot(Particlex1, Particley1)
#plt.plot(Particlex2-Particlex2[0],Particley2-Particley2[0])

plt.plot(Particlex1_end + (Particlex1[0]-Particlex1_end[0]), Particley1_end + (Particley1[0] - Particley1_end[0]))
#plt.plot(Particlex2_end-Particlex2_end[0],Particley2_end-Particley2_end[0])




plt.xlabel('x coordinate [m]')
plt.ylabel('y coordinate [m]')
#plt.legend(['Particle1', 'Particle1_end', 'Particle1_end','Particle2_end'])
plt.grid()
plt.show()

## Data of the gap between the 2 particles

In [ ]:
x = Particlex2 - Particlex1
y = Particley2 - Particley1

cm = plt.cm.get_cmap('winter')
framenumber = range(N)
z = framenumber
sc = plt.scatter(x, y, c=z, vmin=0, vmax=N, s=10, cmap=cm)
plt.colorbar(sc)
#plt.plot(x,y, 'y', linewidth=0.5)
plt.xlabel('x distance between particle centers')
plt.ylabel('y distance between particle centers')
plt.grid()
plt.show()


#plt.figure()

#plt.plot(x,y,'o')
#plt.show()

#fig, ax = plt.subplots()
#im = ax.scatter(x, y, c=c)
#fig.colorbar(im, ax=ax)

print(np.mean(x))
print(np.mean(y))
print(np.max(y))
print(np.min(y))

In [ ]:


x = Particlex2 - Particlex1
y = Particley2 - Particley1
distancelength = np.sqrt(x**2+y**2) 


framenumber = range(N)
sc = plt.scatter(framenumber, distancelength,  s=10)
#sc = plt.scatter(framenumber, y,  s=10)

plt.xlabel('frame number')
plt.ylabel('absolute distance between particle centers')
plt.grid()
plt.show()




#plt.figure()

#plt.plot(x,y,'o')
#plt.show()

#fig, ax = plt.subplots()
#im = ax.scatter(x, y, c=c)
#fig.colorbar(im, ax=ax)

In [ ]:
#x = Particlex2 - Particlex1
#y = Particley2 - Particley1
std_y = np.std(y)
std_distancelength = np.std(distancelength)

print('y distance = ',format(np.mean(y),'.6f'))
print('std y_distance = ',format(np.mean(std_y),'.6f'))
print('absolute distance = ', format(np.mean(distancelength),'.6f'))
print('std absolute distance = ', format(np.mean(std_distancelength),'.6f'))
print(filename)

framenumber = range(N)
sc = plt.scatter(framenumber, distancelength,  s=10)
sc = plt.scatter(framenumber, y,  s=10)

plt.xlabel('frame number')
plt.ylabel('y distance')
plt.grid()
plt.show()

## Save the data to file (optional but can save time later)

In [ ]:
std_yy = np.full((N, 1), std_y)
std_distancelengthh = np.full((N,1), std_distancelength)

DataOut = np.column_stack((framenumber,y,std_yy,distancelength,std_distancelengthh))
#np.savetxt('output.txt', DataOut)
np.savetxt(savename, DataOut, fmt=('%i','%20.8f','%20.8f','%20.8f','%20.8f'))

## Load the data

In [ ]:
framenumber,y,std_y,distancelength,std_distancelength = np.loadtxt(savename, unpack=True)
#print(framenumber)
#print(distancelength)
#print(std_y)